<a href="https://colab.research.google.com/github/uandhafb/postcards-from-my-jungle/blob/main/sound_generation_S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prompts:

1. Birds: granular flutes with radio static.
2. Whales: two low cellos blue water.
3. Giraffes: rhythmic,fast, urban sounds.
4. Elephants:surrealistic, heavy, alternating short/long sounds.
5. Polar Bears:heavy contrabass scraping volcanic rock with metal breath.
6. Komodo Dragons: stereo tremors crossing left/right in impossible geometry.
7. Horses:galloping fast-slow typewriters patterns.
8. Anemone: blurried barroque choir.
9. Spider Monkey:fast patterns built from pedestrian crosswalk beeps.
10. Penguins:staccato ice beats with a hiphop of short vowels.
11. Owls: ritualistic urban sounds.
12. Foxes: collage of heartbeats making music.
13. Dragonfly:prepared piano strings brushed with aluminum foil wind.
14. Paper animals: collage forest, glitchy guitar.




In [ ]:
#Stable Audio Open 1.0 generates variable-length (up to 47s) stereo audio at 44.1kHz from text prompts
#set up HF_TOKEN

!pip install torchsde
import torch
import soundfile as sf
from diffusers import StableAudioPipeline
from IPython.display import Audio

pipe = StableAudioPipeline.from_pretrained("stabilityai/stable-audio-open-1.0", torch_dtype=torch.float16)
pipe = pipe.to("cuda")

# define the prompts
prompt = "staccato ice beats with a hiphop of short vowels."
negative_prompt = "Low quality."

# set the seed for generator
generator = torch.Generator("cuda")

# run the generation
audio = pipe(
    prompt,
    negative_prompt=negative_prompt,
    num_inference_steps=200,
    audio_end_in_s=15.0,
    num_waveforms_per_prompt=3,
    generator=generator,
).audios

output = audio[0].T.float().cpu().numpy()
sf.write("output.wav", output, pipe.vae.sampling_rate)

# Play the audio in Colab
Audio("output.wav")

Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
Flax classes are deprecated and will be removed in Diffusers v1.0.0. We recommend migrating to PyTorch classes or pinning your version of Diffusers.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GatedRepoError: 401 Client Error. (Request ID: Root=1-6993a7b9-611926262d2571693cbdc4af;449f80ab-ff9c-46f4-b1e2-94d2e19d1932)

Cannot access gated repo for url https://huggingface.co/stabilityai/stable-audio-open-1.0/resolve/main/model_index.json.
Access to model stabilityai/stable-audio-open-1.0 is restricted. You must have access to it and be authenticated to access it. Please log in.